In [ ]:
!git clone https://github.com/MIC-DKFZ/nnUNet.git

In [ ]:
!cd nnUNet && git checkout v1.7.0 && pip install -e .

In [ ]:
import os
curr = os.getcwd()
%set_env nnUNet_raw_data_base=$curr/nnUNet_paper
%set_env nnUNet_preprocessed=$curr/nnUNet_preprocessed_paper
%set_env RESULTS_FOLDER=$curr/nnUNet_trained_models

## Download pretrained KiTS21 model

In [12]:
!pip install zenodo_get

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=99b02050b05d8b624114877b059946d642559d7d1290388c6e9f0cb26c675102
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


In [13]:
!zenodo_get 5126443

Title: pretrained nnU-Net weights for KiTS21
Keywords: 
Publication date: 2021-07-23
DOI: 10.5281/zenodo.5126443
Total size: 3372.5 MB

Link: https://zenodo.org/api/files/f08f8382-b8e0-4c2f-8633-8407d8a9188e/nnU-Net_trainingSet_predicted.zip   size: 29.1 MB

Checksum is correct. (d01e8a7f0876150b667aea68873de0e6)

Link: https://zenodo.org/api/files/f08f8382-b8e0-4c2f-8633-8407d8a9188e/Task135_KiTS2021.zip   size: 3343.4 MB

Checksum is correct. (b27ab702742083080b95baac00ba186f)
All files have been downloaded.


In [14]:
!nnUNet_install_pretrained_model_from_zip Task135_KiTS2021.zip



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



In [16]:
import requests
from urllib.parse import urlencode

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/pWEKt6D3qi3-aw'

final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']
response = requests.get(download_url)

dist_path = 'AVUCTK_cases.zip'
with open(dist_path, 'wb') as f:
    f.write(response.content)
    
import zipfile
with zipfile.ZipFile(dist_path, 'r') as zip_ref:
    zip_ref.extractall()

In [17]:
import gzip
import shutil
import os

nn_unet_data_dir = 'kidney_raw'
os.makedirs(nn_unet_data_dir, exist_ok=True)

for case_dir in os.listdir('data'):
  curr_file = os.path.join('data',case_dir,'12.nii.gz')
  shutil.copy2(curr_file,f'{nn_unet_data_dir}/{case_dir}_0000.nii.gz')

### Run inference 

Reads kidney_raw directory and creates masks

In [19]:
!nnUNet_predict -t Task135_KiTS2021 -f 0 -i kidney_raw -o kidney_paper_out_lowres -m 3d_lowres
!nnUNet_predict -t Task135_KiTS2021 -f 0 -i kidney_raw -o kidney_paper_out_fullres -m 3d_fullres
!nnUNet_predict -t Task135_KiTS2021 -f 0 -i kidney_raw -o kidney_paper_out_cascade -m 3d_cascade_fullres



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

using model stored in  /content/nnUNet_trained_models/nnUNet/3d_lowres/Task135_KiTS2021/nnUNetTrainerV2__nnUNetPlansv2.1
This model expects 1 input modalities for each image
Found 9 unique case ids, here are some examples: ['case_8' 'case_4' 'case_8' 'case_6' 'case_8' 'case_4' 'case_2' 'case_1'
 'case_4']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc
number of cases: 9
number of cases that still need to be predicted: 9
emptying cuda cache
loading parameters for folds, [0]
using the following model files:  ['/content/nnUNet_trained_models/nnUNet/3d_lowres/Task135_KiTS2021/nn

# Calculate Dice metrics

In [22]:
import json
import os
import torch
import monai

KIDNEY_DATASET = f'dataset.json'
DATASET_ROOT = os.getcwd()

with open(KIDNEY_DATASET, 'r') as f:
    dataset_json = json.load(f)

validation_with_root = [{
                          'artery':obj['artery'],
                         'vein':obj['vein'],
                         'excret':obj['excret'],
                         'label':obj['label']
                        } for obj in dataset_json['validation']]

load = monai.transforms.LoadImage()
replace_labels = monai.transforms.MapLabelValue(orig_labels=[0,1,2,3,4,5,6], target_labels=[0,0,0,0,2,2,1])
replace_labels_nn = monai.transforms.MapLabelValue(orig_labels=[0,1,2,3], target_labels=[0,1,2,2])
ensure = monai.transforms.EnsureType()
as_discrete = monai.transforms.Compose(
    [
        monai.transforms.AddChannel(), # Channel
    monai.transforms.AsDiscrete(to_onehot=3),
    monai.transforms.AddChannel() # Batch
    ])

In [44]:
ROOT = 'kidney_paper_out_fullres'
dices = []
header = ['Study', 'Kidney', 'Neoplasm']
print('| {:^43} |'.format('Dice metric statistics fullres'))
print('='*47)
print('| {:^6} | {:} | {:} |'.format(*header))

for case in validation_with_root:
    fio = case['label'].split('/')[-2]
    data = load(case['label'])
    data = as_discrete(ensure(replace_labels(data[0]))), data[1]
    pred_data = load(f'{ROOT}/{fio}.nii.gz')
    pred_data = as_discrete(replace_labels_nn(ensure(pred_data[0]))), pred_data[1]
    dice = monai.metrics.compute_meandice(pred_data[0], data[0], False)
    dices.append(dice)
    print('| {:^6} | {:^6.2f} | {:^8.2f} |'.format(*[fio, dice[0,0].cpu().item(), dice[0,1].cpu().item()]))
print('-'*30)
print('| {:^6} | {:^6.2f} | {:^8.2f} |'.format(*['Avg.', torch.cat(dices).mean(0)[0].cpu().item(), torch.cat(dices).mean(0)[1].cpu().item()]))
print('-'*30)
print('| Mean = {:^19.2f} |'.format(torch.cat(dices).mean().cpu().item()))

|       Dice metric statistics fullres        |
| Study  | Kidney | Neoplasm |
| case_1 |  0.71  |   0.92   |
| case_2 |  0.65  |   0.84   |
| case_3 |  0.74  |   0.00   |
| case_4 |  0.75  |   0.92   |
| case_5 |  0.72  |   0.84   |
| case_6 |  0.65  |   0.00   |
| case_7 |  0.73  |   0.00   |
| case_8 |  0.65  |   0.84   |
| case_9 |  0.71  |   0.94   |
------------------------------
|  Avg.  |  0.70  |   0.59   |
------------------------------
| Mean =        0.65         |


In [45]:
ROOT = 'kidney_paper_out_lowres'
dices = []
header = ['Study', 'Kidney', 'Neoplasm']
print('| {:^43} |'.format('Dice metric statistics lowres'))
print('='*47)
print('| {:^6} | {:} | {:} |'.format(*header))

for case in validation_with_root:
    fio = case['label'].split('/')[-2]
    data = load(case['label'])
    data = as_discrete(ensure(replace_labels(data[0]))), data[1]
    pred_data = load(f'{ROOT}/{fio}.nii.gz')
    pred_data = as_discrete(replace_labels_nn(ensure(pred_data[0]))), pred_data[1]
    dice = monai.metrics.compute_meandice(pred_data[0], data[0], False)
    dices.append(dice)
    print('| {:^6} | {:^6.2f} | {:^8.2f} |'.format(*[fio, dice[0,0].cpu().item(), dice[0,1].cpu().item()]))
print('-'*30)
print('| {:^6} | {:^6.2f} | {:^8.2f} |'.format(*['Avg.', torch.cat(dices).mean(0)[0].cpu().item(), torch.cat(dices).mean(0)[1].cpu().item()]))
print('-'*30)
print('| Mean = {:^19.2f} |'.format(torch.cat(dices).mean().cpu().item()))

|        Dice metric statistics lowres        |
| Study  | Kidney | Neoplasm |
| case_1 |  0.49  |   0.00   |
| case_2 |  0.68  |   0.72   |
| case_3 |  0.73  |   0.00   |
| case_4 |  0.78  |   0.21   |
| case_5 |  0.72  |   0.84   |
| case_6 |  0.64  |   0.00   |
| case_7 |  0.75  |   0.00   |
| case_8 |  0.68  |   0.72   |
| case_9 |  0.75  |   0.85   |
------------------------------
|  Avg.  |  0.69  |   0.37   |
------------------------------
| Mean =        0.53         |


In [46]:
ROOT = 'kidney_paper_out_cascade'
dices = []
header = ['Study', 'Kidney', 'Neoplasm']
print('| {:^43} |'.format('Dice metric statistics cascade'))
print('='*47)
print('| {:^6} | {:} | {:} |'.format(*header))

for case in validation_with_root:
    fio = case['label'].split('/')[-2]
    data = load(case['label'])
    data = as_discrete(ensure(replace_labels(data[0]))), data[1]
    pred_data = load(f'{ROOT}/{fio}.nii.gz')
    pred_data = as_discrete(replace_labels_nn(ensure(pred_data[0]))), pred_data[1]
    dice = monai.metrics.compute_meandice(pred_data[0], data[0], False)
    dices.append(dice)
    print('| {:^6} | {:^6.2f} | {:^8.2f} |'.format(*[fio, dice[0,0].cpu().item(), dice[0,1].cpu().item()]))
print('-'*30)
print('| {:^6} | {:^6.2f} | {:^8.2f} |'.format(*['Avg.', torch.cat(dices).mean(0)[0].cpu().item(), torch.cat(dices).mean(0)[1].cpu().item()]))
print('-'*30)
print('| Mean = {:^19.2f} |'.format(torch.cat(dices).mean().cpu().item()))

|       Dice metric statistics cascade        |
| Study  | Kidney | Neoplasm |
| case_1 |  0.51  |   0.33   |
| case_2 |  0.65  |   0.81   |
| case_3 |  0.74  |   0.00   |
| case_4 |  0.77  |   0.33   |
| case_5 |  0.72  |   0.83   |
| case_6 |  0.65  |   0.00   |
| case_7 |  0.73  |   0.00   |
| case_8 |  0.65  |   0.81   |
| case_9 |  0.71  |   0.93   |
------------------------------
|  Avg.  |  0.68  |   0.45   |
------------------------------
| Mean =        0.57         |
